In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import csv
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import sqlite3
import streamlit as st

In [ ]:
def iniciar_driver():
    """Inicia o driver do Chrome."""
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()))

def abrir_site(driver, url):
    """Abre o site especificado."""
    driver.get(url)

def clicar_elemento(driver, by, value, tempo=10):
    """Espera até que o elemento esteja clicável e realiza o clique."""
    elemento = WebDriverWait(driver, tempo).until(
        EC.element_to_be_clickable((by, value))
    )
    elemento.click()

def inserir_texto(driver, by, value, texto, tempo=10):
    """Espera até que o campo esteja visível e insere o texto."""
    campo = WebDriverWait(driver, tempo).until(
        EC.visibility_of_element_located((by, value))
    )
    campo.send_keys(texto)

def preparar_pesquisa(driver):
    """Prepara o campo de entrada principal para pesquisa."""
    # Aguarda o campo de entrada "fake-sbar" estar clicável e clica nele
    clicar_elemento(driver, By.ID, "fake-sbar")
    time.sleep(2)  # Pausa para observar a ação




# Inicializa o driver
driver = iniciar_driver()

# Abre o site
abrir_site(driver, "https://precodahora.ba.gov.br")

# Clica nos botões de navegação e localização
clicar_elemento(driver, By.CSS_SELECTOR, "i.material-icons")  # Botão inicial
time.sleep(3)

clicar_elemento(driver, By.CSS_SELECTOR, "button.btn-danger.nav-location")  # Botão localização
time.sleep(3)

clicar_elemento(driver, By.ID, "add-center")  # Botão "Usar centro do município"
time.sleep(3)

# Insere o nome da cidade e seleciona
inserir_texto(driver, By.CLASS_NAME, "sbar-municipio", "conqu")
time.sleep(2)
clicar_elemento(driver, By.CLASS_NAME, "set-mun")
time.sleep(2)

# Clica no botão 'APLICAR'
clicar_elemento(driver, By.ID, "aplicar")
time.sleep(2)

# Prepara o campo de entrada para as pesquisas de produto
preparar_pesquisa(driver)



2024-10-29 16:03:02.872 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 16:03:03.585 
  command:

    streamlit run c:\Scripts\Projeto\Projeto_scraping_preço\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-10-29 16:03:03.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 16:03:03.589 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 16:03:03.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 16:03:03.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 16:03:03.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 16:03:03.593 

In [3]:
# Loop para pesquisar produtos
produtos = ["lacta"]
dados_produtos = []  # Lista para armazenar os dados dos produtos

for produto in produtos:
    campo_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "top-sbar"))
    )
    campo_input.click()
    campo_input.send_keys(produto)
    time.sleep(random.uniform(1, 3))

    botao_pesquisa = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "i.fa.fa-search"))
    )
    botao_pesquisa.click()
    time.sleep(random.uniform(3, 7))

# O HTML fornecido
html = driver.page_source
# Cria um objeto BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Encontra a div com a classe flex-item2
produtos = soup.find_all('div', class_='flex-item2')

for produto in produtos:
    try:
        preco = float(produto.find_all('div')[1].text.strip().replace('R$', '').replace(',', '.')) if len(produto.find_all('div')) > 1 else 0.0
        codigo_barras = int(produto.find_all('div')[2].text.strip().replace("Código de barras: ", "")) if len(produto.find_all('div')) > 2 else 0
    except ValueError:
        preco = 0.0
        codigo_barras = 0

    dados_produto = {
        'nome': produto.find('strong').text.strip() if produto.find('strong') else "Não informado",
        'preco': preco,
        'codigo_barras': codigo_barras,
        'data_emissao': produto.find_all('div')[3].text.strip() if len(produto.find_all('div')) > 3 else "Não informado",
        'data_extracao': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # Data e hora da extração
        'empresa': produto.find_all('div')[4].text.strip() if len(produto.find_all('div')) > 4 else "Não informado",
        'endereco': produto.find_all('div')[5].text.strip() if len(produto.find_all('div')) > 5 else "Não informado"
    }

    # Exibe os dados do produto extraído
    print(f"\nProduto encontrado:")
    for chave, valor in dados_produto.items():
        print(f"{chave.capitalize()}: {valor}")

    # Adiciona os dados do produto à lista
    dados_produtos.append(dados_produto)

# Cria um DataFrame com os dados coletados
df_produtos = pd.DataFrame(dados_produtos)

# Salva os dados em um arquivo CSV, adicionando ao existente
df_produtos.to_csv('dados_produtos.csv', mode='a', index=False, header=False, encoding='utf-8')

print("\nDados salvos em 'dados_produtos.csv'.")


Produto encontrado:
Nome: BIS LACTA XTRA MAIS CHOC 45G
Preco: 0.0
Codigo_barras: 0
Data_emissao: há 2 dia(s), 4 hora(s), 9 minuto(s) e 28 segundo(s)
Data_extracao: 2024-10-29 15:55:51
Empresa: SUPERMERCADO SAO JOAO
Endereco: RUA PLACIDO DE CASTRO 179 CENTRO 45000830, VITORIA DA CONQUISTA

Produto encontrado:
Nome: BEB LACTA NESTON MACA BAN MAM 280ML
Preco: 0.43
Codigo_barras: 7891000090732
Data_emissao: há 2 dia(s), 4 hora(s), 3 minuto(s) e 10 segundo(s)
Data_extracao: 2024-10-29 15:55:51
Empresa: SUPERMERCADO SAO JOAO
Endereco: RUA PLACIDO DE CASTRO 179 CENTRO 45000830, VITORIA DA CONQUISTA

Produto encontrado:
Nome: CHOCOLATE LACTA 60% CACAU MIX 85G
Preco: 0.0
Codigo_barras: 0
Data_emissao: há 4 hora(s), 31 minuto(s) e 23 segundo(s)
Data_extracao: 2024-10-29 15:55:51
Empresa: SUPERMERCADO SAO JOAO
Endereco: RUA PLACIDO DE CASTRO 179 CENTRO 45000830, VITORIA DA CONQUISTA

Produto encontrado:
Nome: CHOCOLATE LACTA 70% CACAU 85G
Preco: 0.0
Codigo_barras: 0
Data_emissao: há 2 dia(s), 4 